In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import mnist


(x_train, y1_train), (x_test, y1_test) = mnist.load_data()



x1_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x1_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255



# Read A-Z Handwritten dataset
my_data = pd.read_csv('Final_A_Z_handwritten.csv').astype('float32')

# Separate features and labels
x2 = my_data.drop('0', axis=1).values
y2 = my_data['0'].values

# Reshape A-Z images into 28x28 pixels
x2 = x2.reshape(x2.shape[0], 28, 28, 1)

# Normalize A-Z data
x2 = x2 / 255.0

# Split A-Z data into training and test sets
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

y1_train+=26
y1_test+=26


x_train_combined = np.concatenate((x2_train, x1_train), axis=0)
y_train_combined = np.concatenate((y2_train, y1_train), axis=0)
x_test_combined = np.concatenate((x2_test, x1_test), axis=0)
y_test_combined = np.concatenate((y2_test, y1_test), axis=0)

# Convert labels to categorical (one-hot encoding)
num_classes = 36  # 0-9 for digits, 10-35 for A-Z
y_train_combin = to_categorical(y_train_combined, num_classes)
y_test_combin = to_categorical(y_test_combined, num_classes)


# Build the model
my_model = Sequential([  
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPool2D((2, 2), strides=2),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPool2D((2, 2), strides=2),
    Conv2D(128, (3, 3), activation='relu', padding='valid'),
    MaxPool2D((2, 2), strides=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
my_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('FinalTry.keras', save_best_only=False, save_freq='epoch', verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1)


my_model.summary()


c:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 36)             │         4,644 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,468 (540.89 KB)

 Trainable params: 138,468 (540.89 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Train the model
history = my_model.fit(
    x_train_combined, y_train_combin,
    epochs=7, # You can increase this as needed
    validation_data=(x_train_combined, y_train_combin),
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


# Evaluate the model
print("The validation accuracy is:", history.history['val_accuracy'])
print("The training accuracy is:", history.history['accuracy'])
print("The validation loss is:", history.history['val_loss'])
print("The training loss is:", history.history['loss'])


Epoch 1/7
10487/10488 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8887 - loss: 0.3905
Epoch 1: saving model to FinalTry.keras
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 232s 22ms/step - accuracy: 0.8887 - loss: 0.3905 - val_accuracy: 0.9722 - val_loss: 0.0941 - learning_rate: 0.0010
Epoch 2/7
10485/10488 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9732 - loss: 0.0888
Epoch 2: saving model to FinalTry.keras
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 211s 20ms/step - accuracy: 0.9732 - loss: 0.0888 - val_accuracy: 0.9838 - val_loss: 0.0534 - learning_rate: 0.0010
Epoch 3/7
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9801 - loss: 0.0652
Epoch 3: saving model to FinalTry.keras
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 374s 36ms/step - accuracy: 0.9801 - loss: 0.0652 - val_accuracy: 0.9849 - val_loss: 0.0495 - learning_rate: 0.0010
Epoch 4/7
10487/10488 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9835 - loss: 0.0534
Epoch 4: saving model to FinalTry.keras
10488/10488 ━━━━━━━━━━━━━━━━━━━━ 440

KeyboardInterrupt: 